# Prédiction de l'action du joueur de Tennis

**Lire les donées**

In [48]:
# Lecture des données

import numpy as np
import PIL
from PIL import Image 
import glob

# Label de la sortie
# 0 : cd
# 1 : rv
# 2 : sm
# 3 : sv

# 0 : sv j1
# 1 : sv j2
# 2 : cd j1
# 3 : cd j2

# 0 : rv
# 1 : rv_slice
# 2 : rv_vol
# 3 : rv_2_main
# 4 : cd_flat
# 5 : cd_OS
# 6 : cd_slice
# 7 : cd_vol
# 8 : sv_flat 
# 9 : sv_kick
# 10: sv_slice
# 11: sm

action_size = 12
data_size = 100
frame_size = 15
action_list = [0, 1, 2] # [0, 4, 8]
X = []
y = []
for k in action_list: # range(action_size): # chaque action
    for i in range(data_size): # chaque vidéo est une données
        D = []
        for image in glob.glob(f"F:/Tennis/prediction/ac{k}/vid ({i})/*.jpg"): # for j in range(frame_size): # chaque frame est une variable, 16*12 * 15 = 2880
            # img = PIL.Image.open(f"F:/Tennis/prediction/ac{k}/vid ({i})/frame_0{j}.jpg").convert("L")
            img = Image.open(image).convert("L")
            img = img.resize((16, 12))
            imgarr = np.array(img)
            img_row = imgarr.reshape(-1) # transformer en ligne
            D = np.append(D, img_row)
        X.append(D)
        y.append(k)
X = np.array(X)
y = np.array(y)
print(len(X))
print(X.shape, X[20].shape, X[21][2833])
print(type(X), type(X[0]), type(X[0][0]))
print(len(y))
print(y.shape, y[101])
print(type(y), type(y[101]))

300
(300, 2880) (2880,) 0.0
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.float64'>
300
(300,) 1
<class 'numpy.ndarray'> <class 'numpy.int32'>


In [41]:
# lecture des données pour CNN


In [43]:
# Compression des images
# 0 cd
# 1 rv
# 2 sm
# 3 sv

# import cv2

# img = cv2.imread('frame_0.jpg')
# print(img.shape)
# #x, y = img.shape[0:2]
# # cv2.imshow('OriginalPicture', img)
 
# img_test1 = cv2.resize(img, (int(img.shape[0] / 40), int(img.shape[1] / 40)))
# print(img_test1.shape)
# # cv2.imshow('resize0', img_test1)

img = PIL.Image.open("F:/Tennis/prediction/frame_0.jpg").convert("L") # L = R * 299/1000 + G * 587/1000 + B * 114/1000 # ("RGB")
img = img.resize((20, 20))
img = img.save("F:/Tennis/prediction/frame_0_test.jpg")

**K-means**

In [27]:
# from sklearn.cluster import KMeans
# import numpy as np
# import matplotlib.pyplot as plt

# X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
# kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X, y)
# kmeans.labels_
# kmeans.predict([[0, 0], [12, 3]])
# kmeans.cluster_centers_

# plt.scatter(X, y, c=kmeans.labels_)
# plt.show()

**Arbre de décision**

In [28]:
# from sklearn.datasets import load_iris
# from sklearn import tree

# X, y = X.values, y.values
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X, y)

# tree.plot_tree(clf)

**Réseaux de neurones**

In [49]:
# Construction du model avec tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

model = Sequential(
    [               
        tf.keras.Input(shape = 2880), # 16*12 * 15 = 2880
        Dense(units=16, activation="sigmoid"),
        Dense(units=32, activation="sigmoid"),
        Dense(units=3, activation="softmax"),
    ]
)          
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                46096     
                                                                 
 dense_16 (Dense)            (None, 32)                544       
                                                                 
 dense_17 (Dense)            (None, 3)                 99        
                                                                 
Total params: 46,739
Trainable params: 46,739
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Préparation de l'ensemble de test
import numpy as np
import PIL
from PIL import Image 
test_size = 15
frame_size = 15
X_test = []
# [0 0 0 0 0 4 4 4 4 4 8 8 8 8 8]

for i in range(test_size): # chaque vidéo est une données
        T = []
        for image in glob.glob(f"F:/Tennis/prediction/test/test{i}/*.jpg"): # for j in range(frame_size): # chaque frame est une variable, 16*12 * 15 = 2880
            # img = PIL.Image.open(f"F:/Tennis/prediction/test/test{i}/frame_{j}.jpg").convert("L")
            img = Image.open(image).convert("L")
            img = img.resize((16, 12))
            imgarr = np.array(img)
            img_row = imgarr.reshape(-1) # transformer en ligne
            T = np.append(T, img_row)
        X_test.append(T)
X_test = np.array(X_test)
print(X_test.shape)

(15, 2880)


In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

model.compile(
    #optimizer = 'RMSprop', loss = 'categorical_crossentropy', metrics = ['acc']
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), # 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X, y,
    epochs = 50,
    # batch_size = 32 # données transmises pour une session
)

prediction = model.predict(X_test) # np.expand_dims(X_test, axis = 1)
# prediction_p = tf.nn.softmax(prediction)
# yhat = np.argmax(prediction)
print (prediction)

# Softmax
# def softmax(z):  
#     ez = np.exp(z)
#     a = ez/np.sum(ez)
#     return a


Epoch 1/50
10/10 [==============================] - 0s 1ms/step - loss: 1.2495 - accuracy: 0.3367
Epoch 2/50
10/10 [==============================] - 0s 1ms/step - loss: 1.1400 - accuracy: 0.4067
Epoch 3/50
10/10 [==============================] - 0s 1ms/step - loss: 1.1061 - accuracy: 0.4033
Epoch 4/50
10/10 [==============================] - 0s 1ms/step - loss: 1.0932 - accuracy: 0.3867
Epoch 5/50
10/10 [==============================] - 0s 2ms/step - loss: 1.0876 - accuracy: 0.4300
Epoch 6/50
10/10 [==============================] - 0s 2ms/step - loss: 1.0832 - accuracy: 0.4833
Epoch 7/50
10/10 [==============================] - 0s 2ms/step - loss: 1.0815 - accuracy: 0.4333
Epoch 8/50
10/10 [==============================] - 0s 1ms/step - loss: 1.0850 - accuracy: 0.3967
Epoch 9/50
10/10 [==============================] - 0s 2ms/step - loss: 1.0834 - accuracy: 0.4000
Epoch 10/50
10/10 [==============================] - 0s 1ms/step - loss: 1.0750 - accuracy: 0.4033
Epoch 11/50
10/10 [

# [0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8]
# [4, 0, 0, 0, 0, 4, 4, 0, 4, 4, 0, 0, 0, 0, 0]
# 8/15

**Réseau neuronal convolutif**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [54]:
# Modèle général pour tester les paramètres

# def neural_network(num_layer, units, learning_rate=0.01):
#     """
#     Parameters
#     ----------
#     num_layer       : int
#                     number of layers
#     units           : list
#                     number of units in each layer
#     learning_rate   : float
#                     learning rate of the optimizer of the model
#     """
#     if (len(units) != num_layer):
#         raise ValueError("Number of list of units must be equal to number of layers")
#     model = Sequential()
#     model.add(Dense(units=units[0], input_dim=X_train.shape[1], activation="relu"))
#     for i in range(1, num_layer-1):
#         model.add(Dense(units=units[i], activation="relu"))
#     model.add(Dense(units=1, activation="linear"))
#     model.compile(
#         loss=tf.keras.losses.MeanAbsoluteError(),
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#         metrics=[tf.keras.metrics.MeanAbsoluteError()]
#     )
#     return model

In [55]:
# Evaluation des modèle

# histories_neural_network = []
# def neural_network_score(num_layer, units, learning_rate=[0.001, 0.01, 0.1]):
#     """
#     Parameters
#     ----------
#     num_layer       : int
#                     number of layers
#     units           : list
#                     number of units in each layer
#     learning_rate   : list
#                     learning rate of the optimizer of the model
#     """
#     if (len(units) != num_layer):
#         raise ValueError('Le nombre de couches doit être égale à la taille de la liste des neurones')
#     for i in range(len(learning_rate)):
#         model = neural_network(num_layer, units, learning_rate=learning_rate[i])
#         model.save(f"neural_network_model_{i}.h5")
#         history = model.fit(
#             X_train, y_train,
#             epochs=50,
#             batch_size=50,
#             validation_split=0.2
#         )
#         histories_neural_network.append(history)
#         y_pred = model.predict(X_test)
#         y_pred = np.reshape(y_pred, (y_pred.shape[0],))
#         print(f'TEST - R2 score avec le taux {learning_rate[i]} : {r2_score(y_test_numpy, y_pred)}')
#         print(f'TEST - MAE score avec le taux {learning_rate[i]} : {history.history["val_loss"][-1]}')

In [ ]:
# Entraînement des différents modèles

# num_layer = 4
# units = [128, 256, 256, 1]
# neural_network_score(num_layer, units)

In [ ]:
#Visualiser les traces

# from sklearn.metrics import r2_score
# plt.plot(histories_neural_network[0].history['val_loss'])
# plt.plot(histories_neural_network[1].history['val_loss'])
# plt.plot(histories_neural_network[2].history['val_loss'])
# plt.title('Model loss')  
# plt.ylabel('Val Loss')  
# plt.xlabel('Epoch')  
# plt.legend(['taux = 0.001', 'taux = 0.01', 'taux = 0.1'], loc='upper right')  
# plt.show()

In [ ]:
# Test de la performance
#from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#from math import sqrt

# X_test_numpy = np.array(X_test)
# y_test_numpy = np.array(y_test)
# y_pred = model.predict(X_test_numpy)
# y_pred = np.reshape(y_pred, (y_pred.shape[0],))
# plt.scatter(y_test, y_pred, color="b")
# plt.plot(y_test, y_test, color="r")
# r2_nn = r2_score(y_test_numpy, y_pred)
# print('TEST - R2 score - Réseau de neurones: ', r2_nn)